# Tour De France Exploratory Data Analysis

In [1]:
import pandas as pd
import numpy as np
import altair as alt

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/camminady/LeTourDataSet/master/data/TDF_Riders_History.csv")

In [3]:
df = df.drop(columns = ['Unnamed: 0'])

## Correct Known Issues

See issues on data set repository

In [4]:
#remove years that did not use time for GC win
df = df[~df['Year'].isin([1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912])]

In [5]:
df.loc[(df['Year'] == 1903) & (df['Rider'] == 'PIERRE DESVAGES'), 'GapSeconds'] = 212056
df.loc[(df['Year'] == 1903) & (df['Rider'] == 'ARSÈNE MILLOCHEAU'), 'GapSeconds'] = 219450

In [6]:
df.loc[(df['Year'] == 1904) & (df['Rider'] == 'ANTOINE DEFLOTRIERE'), 'GapSeconds'] = 361722

In [7]:
df.loc[(df['Year'] == 1981) & (df['Rider'] == 'BERNARD HINAULT'), 'Times'] = '''96h 19' 38"'''
df.loc[(df['Year'] == 1981) & (df['Rider'] == 'BERNARD HINAULT'), 'TotalSeconds'] = 346778

In [8]:
df.loc[(df['Year'] == 1987) & (df['Rider'] == 'ROBERT FOREST'), 'Times'] = '''117h 2' 46"'''
df.loc[(df['Year'] == 1987) & (df['Rider'] == 'ROBERT FOREST'), 'TotalSeconds'] = 421366

In [9]:
df.loc[(df['Year'] == 1997) & (df['Rider'] == 'MARCO PANTANI'), 'Times'] = '''100h 44' 3"'''
df.loc[(df['Year'] == 1997) & (df['Rider'] == 'MARCO PANTANI'), 'TotalSeconds'] = 362678

In [10]:
# save for re-use
df.to_csv('../data/tdf_cleaned.csv', index = False)

## Plot Winning Time Gaps

### Manually Calculate Time Gaps

In [11]:
dfs = []
for year in df['Year'].unique():
    subset = df[df['Year'] == year]
    winning_time = subset['TotalSeconds'].min()
    
    subset['Time Gap'] = subset['TotalSeconds'] - winning_time
    
    dfs.append(subset)
    
time_gaps = pd.concat(dfs)

/var/folders/49/3f_15lzd6b57lnzlk4m_9k440000gn/T/ipykernel_4889/4071253453.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['Time Gap'] = subset['TotalSeconds'] - winning_time
/var/folders/49/3f_15lzd6b57lnzlk4m_9k440000gn/T/ipykernel_4889/4071253453.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['Time Gap'] = subset['TotalSeconds'] - winning_time
/var/folders/49/3f_15lzd6b57lnzlk4m_9k440000gn/T/ipykernel_4889/4071253453.py:6: SettingWithCopyWarning: 
A value is trying to be set on a c

In [12]:
time_gaps = time_gaps.sort_values(['Year', 'Rank'])

In [13]:
#save for re-use
time_gaps.to_csv('../data/all_time_gaps.csv', index = False)

In [14]:
gaps_df = time_gaps.loc[time_gaps.groupby('Year')['TotalSeconds'].idxmin() + 1]

In [15]:
time_gaps.to_csv('../data/winning_time_gaps.csv', index = False)

In [16]:
gaps_df['3 Year Rolling Average'] = gaps_df['Time Gap'].rolling(window = 3).mean()
gaps_df['5 Year Rolling Average'] = gaps_df['Time Gap'].rolling(window = 5).mean()

Add null values for years the tour wasn't held

In [17]:
ww2_gap = pd.DataFrame(data = {'Year': [i for i in range(1940, 1947)], 
                               'Time Gap':[None, None, None, None, None, None, None]})

In [18]:
no_gc_gap = pd.DataFrame(data = {'Year': [i for i in range(1905, 1913)], 
                                 'Time Gap': [None, None, None, None, None, None, None, None]})


In [19]:
gaps_df = pd.concat([gaps_df, ww2_gap, no_gc_gap])

Melt for easy plotting

In [20]:
gaps_df = gaps_df.rename(columns = {'Time Gap': 'Winning Time Gap'})
molten_gaps_df = pd.melt(gaps_df, id_vars = ['Year'], 
                          value_vars = ['Winning Time Gap', '3 Year Rolling Average', '5 Year Rolling Average'])

In [21]:
molten_gaps_df.columns = ['Year', 'Measure', 'Val']

In [22]:
ww2_gaps_rect = pd.DataFrame({'x1': [1940], 'x2': [1947], 'y1': [0], 'y2': [11000]})
no_gc_gaps_rect = pd.DataFrame({'x1': [1905], 'x2': [1913], 'y1': [0], 'y2': [11000]})

In [23]:
ww2_gap_rect = alt.Chart(ww2_gaps_rect).mark_rect(
    fill = 'lightgray', 
    opacity = 0.5, 
    width = 7
).encode(
    x = alt.X('x1', scale = alt.Scale(domain = [1900, 2023])), 
    x2 = 'x2', 
    y = alt.Y('y1', scale = alt.Scale(domain = [0, 11000])), 
    y2 = 'y2'
)


no_gc_gap_rect = alt.Chart(no_gc_gaps_rect).mark_rect(
    fill = 'lightgray', 
    opacity = 0.5, 
    width = 7
).encode(
    x = alt.X('x1', scale = alt.Scale(domain = [1900, 2023])), 
    x2 = 'x2', 
    y = alt.Y('y1', scale = alt.Scale(domain = [0, 11000])), 
    y2 = 'y2'
)

In [24]:
base = alt.Chart(molten_gaps_df).mark_line(
    size = 2
).encode(
    x = alt.X('Year:Q', 
              axis = alt.Axis(title = 'Year', titleFontSize = 15, format = '4f',
                              tickCount = round(gaps_df['Year'].nunique() / 4)), 
                              scale = alt.Scale(domain = [1900, 2025])), 
    y = alt.Y('Val', axis = alt.Axis(title = 'Time Gap', titleFontSize = 15, format = '1f')), 
    color = alt.Color('Measure', legend = alt.Legend(title = 'Metric')), 
    tooltip = [alt.Tooltip('Year'), alt.Tooltip('Measure', title = 'Measure'), alt.Tooltip('Val', title = 'Value')]
)

In [25]:
tooltips = alt.Chart(molten_gaps_df).mark_line(
    strokeWidth = 30, 
    opacity = 0.01
).encode(
    x = alt.X('Year:Q'),
    y = alt.Y('Val'), 
    color = alt.Color('Measure', legend = None),
    tooltip = [alt.Tooltip('Year'), alt.Tooltip('Measure', title = 'Measure'), alt.Tooltip('Val', title = 'Value')]
)

In [26]:
#to do - note for 1989
c = alt.layer(base, tooltips)

alt.layer(c, ww2_gap_rect, no_gc_gap_rect).properties(
    height = 400, 
    width = 800, 
    title = alt.Title('The Tour de France winning time gap has decreased over time', 
                      fontSize = 25, dx = 10)
).interactive()

alt.LayerChart(...)

## Winning Gap vs Median Gap

In [27]:
median_gap = time_gaps[time_gaps['Time Gap'] != 0].groupby('Year')['Time Gap'].median().reset_index()
median_gap = pd.concat([median_gap, no_gc_gap, ww2_gap])
median_gap = median_gap.rename(columns = {'Time Gap': 'Val'})
median_gap['Measure'] = 'Median Time Gap'

In [28]:
winning_gap = molten_gaps_df[molten_gaps_df['Measure'] == 'Winning Time Gap']

In [29]:
median_v_winning = pd.concat([median_gap, winning_gap])

In [30]:
median_v_winning_chart = alt.Chart(median_v_winning).mark_line(
    size = 2
).encode(
    x = alt.X('Year:Q', axis = alt.Axis(title = 'Year', titleFontSize = 15, format = '4f',
                                        tickCount = round(gaps_df['Year'].nunique() / 4)), 
                                        scale = alt.Scale(domain = [1900, 2025])), 
    y = alt.Y('Val', axis = alt.Axis(title = 'Time Gap', titleFontSize = 15, format = '1f')), 
    color = alt.Color('Measure')
)

In [31]:
alt.layer(median_v_winning_chart, ww2_gap_rect, no_gc_gap_rect).properties(
    height = 500, 
    width = 750, 
    title = alt.Title('Winning and Median Tour de France Time Gaps', 
                      fontSize = 25, dx = 10)
)

alt.LayerChart(...)

In [35]:
median_gap[(median_gap['Year'] > 1960) & (median_gap['Year'] < 1970)]

,Year,Val,Measure
39,1961,5517.0,Median Time Gap
40,1962,7056.5,Median Time Gap
41,1963,4208.5,Median Time Gap
42,1964,6633.0,Median Time Gap
43,1965,3552.0,Median Time Gap
44,1966,94.5,Median Time Gap
45,1967,3436.0,Median Time Gap
46,1968,2578.0,Median Time Gap
47,1969,8779.0,Median Time Gap


In [37]:
time_gaps[time_gaps['Year'] == 1966]

,Rank,Rider,Rider No.,Team,Times,Gap,B,P,Year,Distance (km),Number of stages,TotalSeconds,GapSeconds,ResultType,Time Gap
2616,1,LUCIEN AIMAR,1,FORD-FRANCE-HUTCHINSON,117h 34' 21'',-,NaN,NaN,1966,4329,25,423261,0,time,0
2617,2,JAN JANSSEN,36,PELFORTH-SAUVAGE-LEJEUNE-WOLBER,117h 35' 28'',+ 00h 01' 07'',NaN,NaN,1966,4329,25,423328,67,time,67
2618,3,RAYMOND POULIDOR,26,MERCIER-BP-HUTCHINSON,117h 36' 23'',+ 00h 02' 02'',NaN,NaN,1966,4329,25,423383,122,time,122


## All time gaps

In [32]:
time_gaps_subset = time_gaps[time_gaps['Year'] >= 1990]
time_gaps_subset.shape

(4931, 15)

In [33]:
alt.Chart(time_gaps_subset).mark_point().encode(
    x = alt.X('Year:Q', axis = alt.Axis(title = 'Year', titleFontSize = 15, format = '4f',
                                        tickCount = round(time_gaps_subset['Year'].nunique() / 4)), 
                                        scale = alt.Scale(domain = [1990, 2025])), 
    y = alt.Y('Time Gap')
)

alt.Chart(...)